In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import statistics
from scipy.stats import mode  # Add this line
from flask import Flask, render_template, request
from sklearn.decomposition import PCA

df_train = pd.read_csv("/home/anusheykhan/mysite/Training.csv")
df_test = pd.read_csv("/home/anusheykhan/mysite/Testing.csv")


# Define the symptoms and other necessary variables
symptoms = ['itching','skin_rash','nodal_skin_eruptions','continuous_sneezing','shivering','chills','joint_pain',
    'stomach_pain','acidity','ulcers_on_tongue','muscle_wasting','vomiting','burning_micturition','spotting_ urination','fatigue',
    'weight_gain','anxiety','cold_hands_and_feets','mood_swings','weight_loss','restlessness','lethargy','patches_in_throat',
    'irregular_sugar_level','cough','high_fever','sunken_eyes','breathlessness','sweating','dehydration','indigestion',
    'headache','yellowish_skin','dark_urine','nausea','loss_of_appetite','pain_behind_the_eyes','back_pain','constipation',
    'abdominal_pain','diarrhoea','mild_fever','yellow_urine','yellowing_of_eyes','acute_liver_failure','fluid_overload',
    'swelling_of_stomach','swelled_lymph_nodes','malaise','blurred_and_distorted_vision','phlegm','throat_irritation',
    'redness_of_eyes','sinus_pressure','runny_nose','congestion','chest_pain','weakness_in_limbs','fast_heart_rate',
    'pain_during_bowel_movements','pain_in_anal_region','bloody_stool','irritation_in_anus','neck_pain','dizziness','cramps',
    'bruising','obesity','swollen_legs','swollen_blood_vessels','puffy_face_and_eyes','enlarged_thyroid','brittle_nails',
    'swollen_extremeties','excessive_hunger','extra_marital_contacts','drying_and_tingling_lips','slurred_speech','knee_pain','hip_joint_pain',
    'muscle_weakness','stiff_neck','swelling_joints','movement_stiffness','spinning_movements','loss_of_balance','unsteadiness','weakness_of_one_body_side',
    'loss_of_smell','bladder_discomfort','foul_smell_of urine','continuous_feel_of_urine','passage_of_gases','internal_itching','toxic_look_(typhos)',
    'depression','irritability','muscle_pain','altered_sensorium','red_spots_over_body','belly_pain','abnormal_menstruation','dischromic _patches',
    'watering_from_eyes','increased_appetite','polyuria','family_history','mucoid_sputum','rusty_sputum','lack_of_concentration','visual_disturbances',
    'receiving_blood_transfusion','receiving_unsterile_injections','coma','stomach_bleeding','distention_of_abdomen','history_of_alcohol_consumption',
    'fluid_overload','blood_in_sputum','prominent_veins_on_calf','palpitations','painful_walking','pus_filled_pimples','blackheads','scurring','skin_peeling',
    'silver_like_dusting','small_dents_in_nails','inflammatory_nails','blister','red_sore_around_nose','yellow_crust_ooze']
disease = ['Fungal infection','Allergy','GERD','Chronic cholestasis','Drug Reaction',
        'Peptic ulcer diseae','AIDS','Diabetes','Gastroenteritis','Bronchial Asthma','Hypertension',
        ' Migraine','Cervical spondylosis',
        'Paralysis (brain hemorrhage)','Jaundice','Malaria','Chicken pox','Dengue','Typhoid','hepatitis A',
'Hepatitis B','Hepatitis C','Hepatitis D','Hepatitis E','Alcoholic hepatitis','Tuberculosis',
'Common Cold','Pneumonia','Dimorphic hemmorhoids(piles)',
'Heartattack','Varicoseveins','Hypothyroidism','Hyperthyroidism','Hypoglycemia','Osteoarthristis',
'Arthritis','(vertigo) Paroymsal  Positional Vertigo','Acne','Urinary tract infection','Psoriasis',
'Impetigo']
temp=[]
for x in range(0,len(symptoms)):
    temp.append(0)

# Encoding the disease from the training data set
df_train.replace({'prognosis':{'Fungal infection':0,'Allergy':1,'GERD':2,'Chronic cholestasis':3,'Drug Reaction':4,
'Peptic ulcer diseae':5,'AIDS':6,'Diabetes ':7,'Gastroenteritis':8,'Bronchial Asthma':9,'Hypertension ':10,
'Migraine':11,'Cervical spondylosis':12,
'Paralysis (brain hemorrhage)':13,'Jaundice':14,'Malaria':15,'Chicken pox':16,'Dengue':17,'Typhoid':18,'hepatitis A':19,
'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,'Alcoholic hepatitis':24,'Tuberculosis':25,
'Common Cold':26,'Pneumonia':27,'Dimorphic hemmorhoids(piles)':28,'Heart attack':29,'Varicose veins':30,'Hypothyroidism':31,
'Hyperthyroidism':32,'Hypoglycemia':33,'Osteoarthristis':34,'Arthritis':35,
'(vertigo) Paroymsal  Positional Vertigo':36,'Acne':37,'Urinary tract infection':38,'Psoriasis':39,
'Impetigo':40}}, inplace=True)

df_test.replace({'prognosis':{'Fungal infection':0,'Allergy':1,'GERD':2,'Chronic cholestasis':3,'Drug Reaction':4,
'Peptic ulcer diseae':5,'AIDS':6,'Diabetes ':7,'Gastroenteritis':8,'Bronchial Asthma':9,'Hypertension ':10,
'Migraine':11,'Cervical spondylosis':12,
'Paralysis (brain hemorrhage)':13,'Jaundice':14,'Malaria':15,'Chicken pox':16,'Dengue':17,'Typhoid':18,'hepatitis A':19,
'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,'Alcoholic hepatitis':24,'Tuberculosis':25,
'Common Cold':26,'Pneumonia':27,'Dimorphic hemmorhoids(piles)':28,'Heart attack':29,'Varicose veins':30,'Hypothyroidism':31,
'Hyperthyroidism':32,'Hypoglycemia':33,'Osteoarthristis':34,'Arthritis':35,
'(vertigo) Paroymsal  Positional Vertigo':36,'Acne':37,'Urinary tract infection':38,'Psoriasis':39,
'Impetigo':40}}, inplace=True)

# Split the data into features (X) and target (y)
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values
num_components = 100# You can choose the number of components you want to keep
pca = PCA(n_components=num_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)



svmm = SVC()
svmm.fit(X_train_pca, y_train)
gnbb = GaussianNB()
gnbb.fit(X_train_pca, y_train)
rff = RandomForestClassifier(max_features=num_components)
rff.fit(X_train_pca, y_train)

# Function to calculate the mode prediction of the classifiers
# Function to calculate the mode prediction of the classifiers
# Function to calculate the mode prediction of the classifiers
# Function to calculate the mode prediction of the classifiers
def modeofclassifiers(classifier1_prediction, classifier2_prediction, classifier3_prediction):
    predictions = np.array([classifier1_prediction, classifier2_prediction, classifier3_prediction])
    mode_prediction, count = mode(predictions, axis=0)
    mode_prediction = mode_prediction.tolist()  # Convert array to list
    if count.size > 1:
        # Multiple modes exist, return the first one
        return mode_prediction[0]
    else:
        return mode_prediction




# Function to predict the disease based on symptoms
def prediction(s1, s2, s3, s4, s5):
    psymptoms = [s1, s2, s3, s4, s5]
    temp = [0] * len(symptoms)

    for k in range(0, len(symptoms)):
        for z in psymptoms:
            if z == symptoms[k]:
                temp[k] = 1
    inputtest = [temp]
    inputtest_pca = pca.transform(inputtest)

    predictnbb = gnbb.predict(inputtest_pca)
    predictsvmm = svmm.predict(inputtest_pca)
    predictrff = rff.predict(inputtest_pca)
    predictedd = modeofclassifiers(predictnbb, predictsvmm, predictrff)
    disease_array = np.array(disease)

    if isinstance(predictedd, int):
        print(disease_array[predictedd])
    elif isinstance(predictedd, list):
        for prediction in predictedd:
            displayedd = disease_array[prediction]
        print(displayedd)
        return displayedd


prediction('runny_nose','dark_urine','nausea','weakness_in_limbs','fast_heart_rate')
import csv

def find_descriptions(filename, input_value):
    descriptions = []
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[0] == input_value:
                descriptions.append(row[1])
    return descriptions

app = Flask(__name__)

# Route for the home page
@app.route('/')
def home():
    return render_template('index.html')

# Route to handle the form submission
@app.route('/predict', methods=['POST'])
def predict():
    symptoms = request.json
    # symptom1 = request.form['symptom1']
    # symptom2 = request.form['symptom2']
    # symptom3 = request.form['symptom3']
    # symptom4 = request.form['symptom4']
    # symptom5 = request.form['symptom5']

    # Perform the prediction
    inputvalue = prediction(symptoms[0], symptoms[1], symptoms[2], symptoms[3], symptoms[4])

    # Fetch the descriptions from the categorized.csv file
    matching_descriptions =find_descriptions('/home/anusheykhan/mysite/categorized.csv', inputvalue[0])

    # return render_template('result.html', inputvalue=inputvalue, descriptions=matching_descriptions)

    return render_template('result.html', inputvalue=inputvalue[0], descriptions=matching_descriptions)

if __name__ == '__main__':
    app.run(debug=True,port= 5002)